# <center>How To Win Udemy</center>

### Sources and credits for this project

- https://www.kaggle.com/andrewmvd/udemy-courses
- Dataset auther: Willden Chase from Concept Center.

In [1]:
# move "Sources and credits for this project" to the end

## Introduction and Goal definition

This dataset contains 3.682 records of courses from 4 subjects (Business Finance, Graphic Design, Musical Instruments and Web Design) taken from Udemy.
Udemy is a massive online open course (MOOC) platform that offers both free and paid courses. Anybody can create a course, a business model by which allowed Udemy to have hundreds of thousands of courses.
This version modifies column names, removes empty columns and aggregates everything into a single csv file for ease of use.

We can draw immidietly some obvious questions realated to the topic like:
- What are the best free courses by subject?
- What are the most popular courses?
- What are the most engaging courses?
- How are courses related?
- Which courses offer the best cost benefit?

While those questions are defentliy worth exploring as details, I more intrested in exploring the question that wraps them all - what makes a course successful? or in other words, how to win Udemy?

In [ ]:
# add - how im I going to answer that question?

### Workflow stages

1. Prepare and clean the data for work. 
2. Analyze, identify patterns, and explore the data.
3. Division of the records into a training group and a test group.
4. Models training and evaluation.

## 1) Prepare and clean the data for work

In [ ]:
# some basic imports that will surely be useful
import pandas as pd
import numpy  as np
import matplotlib.pyplot as plt
